In [24]:
from __future__ import print_function
import sys
print(sys.version)
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d'))

3.5.2 |Anaconda 4.2.0 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
2017-08-21


## CryptoAssets Blockchain
A very simple blockchain that serves the purpose of verifying asset ownership.

Blog post: https://medium.com/@agalea91/representing-an-asset-on-the-blockchain-with-python-2bd9aff7b500

In [25]:
import hashlib
import datetime
import json

In [26]:
class CryptoAssets():
    def __init__(self):
        self.blockchain = [
            self.make_genesis_block()
        ]
        self.next_block_index = 1
#         self.public_ledger = {}
        self.public_ledger = {
            'Alice': {'Tesla Model 3': 1, '200 shares of TSLA': 1},
            'James': {'9425 Sunset Blvd, Beverly Hills, CA 90210': 1}
        }
        self.transactions = []

    def mine(self):
        '''
        Mine a single block by hashing the transactions
        in the cue.
        '''
        timestamp = str(datetime.datetime.now())
        transactions = self.transactions
        prev_block = self.blockchain[-1]

        new_block = Block(
            self.next_block_index,
            timestamp,
            transactions,
            prev_block.hash_id
        )
        self.blockchain.append(new_block)
        self.update_public_ledger(transactions)
        
        self.transactions = []
        self.next_block_index += 1
        
    def make_genesis_block(self):
        '''
        Make the first block. This is done in an arbitray way.
        '''
        block = Block(
            index_number=0,
            timestamp=str(datetime.datetime.now()),
            transactions='',
            prev_block_hash_id=''
        )
        return block
        
    def update_public_ledger(self, transactions):
        '''
        Update dictionary of names and assets as key-value pairs
        '''
        for t in transactions:
            # Remove the asset from the sender
            if t.sender not in self.public_ledger.keys():
                self.public_ledger[t.sender] = {}
            sender = self.public_ledger[t.sender]
            try:
                sender[t.asset] -= 1
            except KeyError:
                sender[t.asset] = -1
            if sender[t.asset] == 0:
                del sender[t.asset]

            # Add the asset to the buyer
            if t.buyer not in self.public_ledger.keys():
                self.public_ledger[t.buyer] = {}
            buyer = self.public_ledger[t.buyer]
            try:
                buyer[t.asset] += 1
            except KeyError:
                buyer[t.asset] = 1

In [27]:
class Block():
    def __init__(self, index_number, timestamp,
                 transactions, prev_block_hash_id):
        # Set the block attributes
        self.index_number = index_number
        self.timestamp = timestamp
        self.transactions = transactions
        self.prev_block_hash_id = prev_block_hash_id
        
        # Create block hash
        self.hash_id = hashlib.sha256(
            '{}-{}-{}-{}'.format(
                self.index_number,
                self.timestamp,
                self.transactions,
                self.prev_block_hash_id
            ).encode('utf-8')
        ).hexdigest()
        
    def __repr__(self):
        return 'Block #{} at {}'.format(self.index_number, self.timestamp)

In [28]:
class Transaction():
    def __init__(self, sender, buyer, asset):
        self.sender = sender
        self.buyer = buyer
        self.asset = asset
        
    def __repr__(self):
        return '{} ({} -> {})'.format(self.asset, self.sender, self.buyer)

In [29]:
crypto_assets = CryptoAssets()

In [30]:
print(json.dumps(
        crypto_assets.public_ledger,
        indent=2
    ))

{
  "Alice": {
    "Tesla Model 3": 1,
    "200 shares of TSLA": 1
  },
  "James": {
    "9425 Sunset Blvd, Beverly Hills, CA 90210": 1
  }
}


In [31]:
crypto_assets.transactions

[]

In [32]:
crypto_assets.transactions.extend([
    Transaction(
        sender='Alice',
        buyer='Bob',
        asset='Tesla Model 3'
    ),
    Transaction(
        sender='Alice',
        buyer='James',
        asset='200 shares of TSLA'
    )
])

In [33]:
crypto_assets.transactions

[Tesla Model 3 (Alice -> Bob), 200 shares of TSLA (Alice -> James)]

In [34]:
crypto_assets.mine()

In [35]:
print(json.dumps(
        crypto_assets.public_ledger,
        indent=2
    ))

{
  "Bob": {
    "Tesla Model 3": 1
  },
  "Alice": {},
  "James": {
    "200 shares of TSLA": 1,
    "9425 Sunset Blvd, Beverly Hills, CA 90210": 1
  }
}


In [36]:
crypto_assets.blockchain

[Block #0 at 2017-08-21 23:14:31.024585,
 Block #1 at 2017-08-21 23:14:31.074297]

In [37]:
crypto_assets.transactions.extend([
    Transaction(
        sender='Bob',
        buyer='James',
        asset='Tesla Model 3'
    ),
    Transaction(
        sender='James',
        buyer='Alice',
        asset='9425 Sunset Blvd, Beverly Hills, CA 90210'
    )
])

In [38]:
crypto_assets.mine()

In [39]:
print(json.dumps(
        crypto_assets.public_ledger,
        indent=2
    ))

{
  "Bob": {},
  "Alice": {
    "9425 Sunset Blvd, Beverly Hills, CA 90210": 1
  },
  "James": {
    "Tesla Model 3": 1,
    "200 shares of TSLA": 1
  }
}


In [40]:
crypto_assets.blockchain

[Block #0 at 2017-08-21 23:14:31.024585,
 Block #1 at 2017-08-21 23:14:31.074297,
 Block #2 at 2017-08-21 23:14:31.114341]

In [41]:
import pandas as pd
df = pd.DataFrame([(block.transactions, block.hash_id) for block in crypto_assets.blockchain],
                  index=[block.index_number for block in crypto_assets.blockchain],
                  columns=['Transactions', 'Block Hash'])
df.index.name = 'Block Index'
df

,Transactions,Block Hash
Block Index,,
0,,2431dd05619e0ad6cedd3d80a63c4cec4ae0cc832c080f...
1,"[Tesla Model 3 (Alice -> Bob), 200 shares of T...",cb42342039e209c601d5f127b060828868d39fb545c61c...
2,"[Tesla Model 3 (Bob -> James), 9425 Sunset Blv...",2a48ed83860e85b34d50b28906fe726f2458d4af9f4610...


In [42]:
from IPython.display import HTML 

df = pd.DataFrame([(block.index_number, block.transactions, block.hash_id) for block in crypto_assets.blockchain],
                  columns=['Index Number', 'Transactions', 'Block Hash'])
HTML(df.to_html(index=False))

Index Number,Transactions,Block Hash
0,,2431dd05619e0ad6cedd3d80a63c4cec4ae0cc832c080f...
1,"[Tesla Model 3 (Alice -> Bob), 200 shares of T...",cb42342039e209c601d5f127b060828868d39fb545c61c...
2,"[Tesla Model 3 (Bob -> James), 9425 Sunset Blv...",2a48ed83860e85b34d50b28906fe726f2458d4af9f4610...


---
Let's say Alice claims that she never sold the car to Bob. Bob can reference their transaction on the blockchain (in block #1). He can verify it's authenticity by confirming the hash for that block. In this case the calculation would go as follows:

---

In [43]:
real_transactions = [
    Transaction(
        sender='Alice',
        buyer='Bob',
        asset='Tesla Model 3'
    ),
    Transaction(
        sender='Alice',
        buyer='James',
        asset='200 shares of TSLA'
    )
]

In [44]:
hashlib.sha256(
    '{}-{}-{}-{}'.format(
        crypto_assets.blockchain[1].index_number,
        crypto_assets.blockchain[1].timestamp,
        real_transactions,
        crypto_assets.blockchain[1].prev_block_hash_id
    ).encode('utf-8')
).hexdigest()

'cb42342039e209c601d5f127b060828868d39fb545c61c4cc1ae157f5c5b791d'

---
Indeed the hash matches, hence the transaction has been verified. To really hammer home the point to Alice, Bob could also find out what the hash would have been if she indeed did not transfer the car. He would calculate this as follows:

---

In [45]:
claimed_transactions = [
    Transaction(
        sender='Alice',
        buyer='James',
        asset='200 shares of TSLA'
    ),
    Transaction(
        sender='James',
        buyer='Alice',
        asset='9425 Sunset Blvd, Beverly Hills, CA 90210'
    )
]

In [46]:
hashlib.sha256(
    '{}-{}-{}-{}'.format(
        crypto_assets.blockchain[1].index_number,
        crypto_assets.blockchain[1].timestamp,
        claimed_transactions,
        crypto_assets.blockchain[1].prev_block_hash_id
    ).encode('utf-8')
).hexdigest()

'acb77092c9393e6119e59250e34d50409007ccd7cc85c9099d8869155b29b8bc'